In [1]:
import sys
from pathlib import Path
from os.path  import join

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

g:\Work\DS\mimolm\notebooks


In [2]:
from src.config import DATA_DIR

ARGO_SCENE_DIR = DATA_DIR / "raw" / "temp"
NUM_SCENARIOS = 1

all_scenario_files = sorted(ARGO_SCENE_DIR.rglob("*.parquet"))
scenario_file_list = (
    all_scenario_files[:NUM_SCENARIOS]
)

2025-01-16 15:24:30.800 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm


In [19]:
from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap

scenario_path = scenario_file_list[0]
scenario_id = scenario_path.stem.split("_")[-1]
static_map_path = (
    scenario_path.parents[0] / f"log_map_archive_{scenario_id}.json"
)

scenario = scenario_serialization.load_argoverse_scenario_parquet(scenario_path)
static_map = ArgoverseStaticMap.from_json(static_map_path)

In [9]:
scenario.tracks

[Track(track_id='89108', object_states=[ObjectState(observed=True, timestep=0, position=(1924.362304993518, 619.2383415029709), heading=-2.4444851993562335, velocity=(-11.223503072020154, -9.37422533125003)), ObjectState(observed=True, timestep=1, position=(1923.7290628806409, 618.7174979179755), heading=-2.4439781666034204, velocity=(-11.20954057161014, -9.363317792427443)), ObjectState(observed=True, timestep=2, position=(1922.9648580599842, 618.0878101130446), heading=-2.444102100056339, velocity=(-11.19669168654929, -9.38140188271227)), ObjectState(observed=True, timestep=3, position=(1922.0777848729458, 617.354265850537), heading=-2.4452144227293404, velocity=(-11.19166051671746, -9.401227686119325)), ObjectState(observed=True, timestep=4, position=(1921.0778154681961, 616.53087555361), heading=-2.4460159034992506, velocity=(-11.200247623301216, -9.426423802277949)), ObjectState(observed=True, timestep=5, position=(1919.9808009379078, 615.6263101085618), heading=-2.446782275468863

In [21]:
static_map.vector_lane_segments[248399542]

LaneSegment(id=248399542, is_intersection=False, lane_type=<LaneType.VEHICLE: 'VEHICLE'>, right_lane_boundary=Polyline(waypoints=[Point(x=359.38, y=-1495.45, z=-12.21), Point(x=353.78, y=-1513.58, z=-11.79)]), left_lane_boundary=Polyline(waypoints=[Point(x=365.71, y=-1495.8, z=-12.08), Point(x=358.73, y=-1515.95, z=-11.83)]), right_mark_type=<LaneMarkType.NONE: 'NONE'>, left_mark_type=<LaneMarkType.DASHED_WHITE: 'DASHED_WHITE'>, predecessors=[248399815, 248399807], successors=[], right_neighbor_id=None, left_neighbor_id=248399580)

In [3]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim")
data_module.setup(stage="fit")
dl = data_module.train_dataloader()

c:\Users\Harshavardhan Patil\.virtualenvs\mimolm-kv68Yv56\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dl.dataset[0]

{'episode_idx': 24,
 'agent/valid': array([[ True,  True,  True, ..., False, False,  True],
        [ True,  True,  True, ..., False, False,  True],
        [ True,  True,  True, ..., False, False,  True],
        ...,
        [ True,  True, False, ...,  True, False,  True],
        [ True,  True, False, ...,  True, False,  True],
        [ True,  True, False, ..., False, False,  True]]),
 'agent/pos': array([[[ 41.436222,  40.630806],
         [152.6057  ,  58.36563 ],
         [ 16.003654,  64.06812 ],
         ...,
         [  0.      ,   0.      ],
         [  0.      ,   0.      ],
         [ 30.30379 ,  40.181934]],
 
        [[ 41.726387,  40.661655],
         [153.11967 ,  58.405014],
         [ 16.0138  ,  64.067665],
         ...,
         [  0.      ,   0.      ],
         [  0.      ,   0.      ],
         [ 30.666174,  40.212708]],
 
        [[ 42.086044,  40.70045 ],
         [153.76477 ,  58.455368],
         [ 15.994055,  64.066185],
         ...,
         [  0.      , 

In [11]:
import h5py

with h5py.File(DATA_DIR / "interim" / "train.h5", "r", libver="latest", swmr=True) as hf:
    print(hf['11'])

<HDF5 group "/11" (2 members)>
